## Initiate the Toolbox
- the parameters set here will be used to collect command-specific data when the `Fetcher` executes the TET pattern.

In [2]:
%load_ext autoreload
%autoreload 2

from humbldata.toolbox.toolbox_controller import Toolbox

toolbox = Toolbox(
    symbol=["AMD", "AAPL"],
    interval="1d",
    start_date="2000-01-01",
    end_date="2021-02-20",
)
toolbox

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Toolbox(symbol=['AMD', 'AAPL'], interval=1d, start_date=2000-01-01, end_date=2021-02-20, provider=yfinance)

In [3]:
toolbox

Toolbox(symbol=['AMD', 'AAPL'], interval=1d, start_date=2000-01-01, end_date=2021-02-20, provider=yfinance)

In [10]:
result = toolbox.technical.mandelbrot_channel(historical=False, window="1m")



In [11]:
result

HumblObject

id: 066703ee-4fa9-7b7b-8000-039b8cfc4cb7
results: {"IR":{"version":12201,"dsl":{"Select":{"expr":[{"Column":"date"},{"Column...
equity_data: {"IR":{"version":11695,"dsl":{"MapFunction":{"input":{"DataFrameScan":...
provider: yfinance
warnings: None
chart: None
extra: {}
context_params: {'symbol': ['AMD', 'AAPL'], 'interval': '1d', 'start_date': '2000-0...
command_params: {'window': '1mo', 'rv_adjustment': True, 'rv_method': 'std', 'rs_me...

In [12]:
result.to_polars()

date,symbol,bottom_price,recent_price,top_price
date,str,f64,f64,f64
2021-02-19,"""AAPL""",127.5741,127.643967,144.8156
2021-02-19,"""AMD""",84.2072,89.580002,96.5083
